In [299]:
import requests
import json
import pandas as pd
import time

# Running flask app from powershell
# $env:FLASK_APP = "api"
# $env:FLASK_ENV = "development"
# flask run

# Running flask from terminal on OSX
# export FLASK_APP=api
# export FLASK_ENV=development
# flask run

# Running with gunicorn (heroku, digitalocean, etc):
# Procfile >
# web: gunicorn wsgi --log-file -
# wsgi.py >
# from api import app as application
#
#if __name__ == "__main__":
#    application.run()

# https://devcenter.heroku.com/articles/getting-started-with-python
# https://www.digitalocean.com/community/tutorials/how-to-serve-flask-applications-with-gunicorn-and-nginx-on-ubuntu-14-04

In [300]:
# Set up heroku database > 
# heroku addons:create heroku-postgresql:hobby-dev
# Postgres db now up
# Look at config vars (should have DATABASE_URL) >
# heroku config
# Connect to remote psql > 
# heroku pg:psql
#


In [301]:
#  baseurl = 'https://vast-depths-89393.herokuapp.com/'
baseurl = 'https://glacial-hollows-90395.herokuapp.com/'
#baseurl = 'http://127.0.0.1:5000/'

In [302]:
url = baseurl +'createUser'
headers = {'content-type': 'application/json'}
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [303]:
response.json()

{'signingKey_hex': '6adb406a079b6105ee7696669549d02b8ddd0050afd651d236d1b1134d25b67c',
 'traderId': '1',
 'verifyKey_hex': 'd7a2da8288c65fa575dda10309ed084dc7bae13035115b245d67aae75900e42b'}

In [304]:
url = baseurl + 'createMarket'

mkRtId = 6
mkId=1
# copy from response
sk = response.json()['signingKey_hex']
vk = response.json()['verifyKey_hex']
tId = response.json()['traderId']

In [305]:
content_makemarket = {"signingKey_hex": sk,
                    "traderId": tId, 
                    "verifyKey_hex": vk,
                    "marketRootId": mkRtId, 
                    "marketBranchId": 1, 
                    "marketMin": 0,
                    "marketMax":1}

response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)

In [306]:
pd.DataFrame(response.json(), index=[0])

,checks,marketBranchId,marketMax,marketMin,marketRootId,traderId
0,True,1,1,0,6,1


In [307]:
# Create a trade
url = baseurl+'createTrade'

content_maketrade = {"signingKey_hex": sk,
                     "traderId": tId,
                     "verifyKey_hex": vk,
                     "marketId": mkId,
                     "price": 0.55,
                     "quantity":1}

In [308]:
# Post trade
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [309]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': 1,
 'traderId': '1'}

In [310]:
# View json response
pd.DataFrame(response.json(), index=[0])


,checks,marketId,price,quantity,traderId
0,True,1,0.55,1,1


In [311]:
# Create a matched trade
url = baseurl+'createTrade'
content_maketrade['quantity'] = -1
response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)

In [312]:
response.json()

{'checks': 'True',
 'marketId': 1,
 'price': 0.55,
 'quantity': -1,
 'traderId': '1'}

In [313]:
# Create many matched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = -1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    content_maketrade['quantity']=1
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('matched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


matched trade: 0 in 4.360753059387207 seconds.
matched trade: 1 in 4.230324983596802 seconds.
matched trade: 2 in 5.051490068435669 seconds.
matched trade: 3 in 4.2892820835113525 seconds.
matched trade: 4 in 4.413121938705444 seconds.
matched trade: 5 in 3.8457767963409424 seconds.
matched trade: 6 in 4.192300796508789 seconds.
matched trade: 7 in 5.122128009796143 seconds.
matched trade: 8 in 4.568428039550781 seconds.
matched trade: 9 in 4.128752946853638 seconds.


In [314]:
url = baseurl+'createMarket'

for i in range(4):
    t = time.time()

    content_makemarket = {"signingKey_hex": sk,
      "traderId": int(tId), "verifyKey_hex": vk,
      "marketRootId": i+1, "marketBranchId": 1, "marketMin": 0, "marketMax":0.7}

    response = requests.post(url, data=json.dumps(content_makemarket), headers=headers)
    if response.json()['checks'] == 'True':
        print('market created: ' + str(i+1) + ' in ' + str(time.time() - t) + ' seconds.')
    else:
        print('failed to create market ' +str(i+1))
    

market created: 1 in 2.686424970626831 seconds.
market created: 2 in 3.749390125274658 seconds.
market created: 3 in 4.742095947265625 seconds.
market created: 4 in 6.976371765136719 seconds.


In [315]:
response.json()

{'checks': 'True',
 'marketBranchId': 1,
 'marketMax': 0.7,
 'marketMin': 0,
 'marketRootId': 4,
 'traderId': 1}

In [316]:
url = baseurl+'viewMarketBounds'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)



In [317]:
a = pd.read_json(response.json())
a

,marketId,marketRootId,marketBranchId,marketMin,marketMax
0,1,6,1,0,1.0
1,2,1,1,0,0.7
2,3,2,1,0,0.7
3,4,3,1,0,0.7
4,5,4,1,0,0.7


In [332]:
url = baseurl+'viewOrderBook'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [333]:
a = pd.read_json(response.json())
a

,marketId,price,quantity,traderId,iMatched
0,1,0.55,1,1,True
1,1,0.55,-1,1,True
10,1,0.55,-1,1,True
11,1,0.55,1,1,True
12,1,0.55,-1,1,True
13,1,0.55,1,1,True
14,1,0.55,-1,1,True
15,1,0.55,1,1,True
16,1,0.55,-1,1,True
17,1,0.55,1,1,True


In [320]:
# Create many unmatched trades
url = baseurl+'createTrade'

for i in range(10):
    t = time.time()
    content_maketrade['quantity'] = 1
    content_maketrade['price'] = 0.44
    response = requests.post(url, data=json.dumps(content_maketrade), headers=headers)
    print('unmatched trade: ' + str(i) + ' in ' + str(time.time() - t) + ' seconds.')


unmatched trade: 0 in 6.311537742614746 seconds.
unmatched trade: 1 in 6.982812881469727 seconds.
unmatched trade: 2 in 1.6550450325012207 seconds.
unmatched trade: 3 in 1.6011948585510254 seconds.
unmatched trade: 4 in 1.8852572441101074 seconds.
unmatched trade: 5 in 1.622169017791748 seconds.
unmatched trade: 6 in 1.577888011932373 seconds.
unmatched trade: 7 in 1.6996738910675049 seconds.
unmatched trade: 8 in 1.5977988243103027 seconds.
unmatched trade: 9 in 1.5711920261383057 seconds.


In [334]:
url = baseurl+'viewOpenTrades'

content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [335]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,0.44,2,1


In [336]:
url = baseurl+'viewMatchedTrades'
content = {}
response = requests.post(url, data=json.dumps(content), headers=headers)

In [337]:
pd.read_json(response.json())

,marketId,price,quantity,traderId
0,1,0.55,0,1
